In [1]:
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [2]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import wget

In [4]:
complete_zip = wget.download(complete_dataset_url)
small_zip = wget.download(small_dataset_url)

In [5]:
import zipfile

In [6]:
#with zipfile.ZipFile('/content/ml-latest-small.zip', "r") as z:
with zipfile.ZipFile('ml-latest-small.zip', "r") as z:
    z.extractall('')

In [7]:
#with zipfile.ZipFile('/content/ml-latest.zip', "r") as z:
with zipfile.ZipFile('ml-latest.zip', "r") as z:
    z.extractall('')

In [8]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:

# from pyspark.sql import SparkSession


# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName("Hello Pyspark") \
#     .config("spark.some.config.option", "some-value") \
#     .getOrCreate()

In [10]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [11]:
#raw_data_test = spark.read.text("/content/ml-latest-small/ratings.csv")
#small_ratings_raw_data = sc.textFile("/content/ml-latest-small/ratings.csv")
small_ratings_raw_data = sc.textFile("ml-latest-small/ratings.csv")

In [12]:
small_ratings_raw_data

/content/ml-latest-small/ratings.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [14]:
small_ratings_raw_data.take(1)[0]

'userId,movieId,rating,timestamp'

In [20]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data.take(1)[0]).map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2]))

In [23]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=99)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [25]:
from pyspark.mllib.recommendation import ALS
import math

seed = 99
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print('For rank',rank,'the RMSE is',error)
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank', best_rank)

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/rdd.py", line 356, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063

PicklingError: ignored